In [2]:
import pymysql
import logging
import pandas as pd
logging.basicConfig(level=logging.INFO)

mysql_host = "142.132.134.21"
mysql_prot = 9030
mysql_user = "dw"
mysql_pd = "a64imrJwqKZc9jiU!*aCYi.xnh9*TVhhBmZi"
mysql_db = "dw"
connection = pymysql.connect(host=mysql_host, user=mysql_user, password=mysql_pd, db=mysql_db, port=mysql_prot, charset='utf8')
cursor = connection.cursor()
logging.info("connection created")


def getSqlResult(sql):
    cursor.execute(sql)
    res_rows = cursor.fetchall()
    df = pd.DataFrame(res_rows)
    return df

INFO:root:connection created


# 0x958236266991bc3fe3b77feaacea120f172c0708ad01c7a715b255f218f9313c

In [17]:
txn_hash = '0x958236266991bc3fe3b77feaacea120f172c0708ad01c7a715b255f218f9313c'
txn_detail_sql = """
select a.block_number,a.log_index,a.transaction_hash
    ,a.token_address,b.symbol,b.token_description,b.decimals
    ,a.from_address,c.tag_code
    ,a.to_address,c.tag_code
    ,a.value
    ,a.value/POWER(10, b.decimals) as txn_value
from prod.transfer_event_eth as a
left join prod.token as b
on a.token_address = b.address
left join
(
    select address
        ,max(tag_code) as tag_code
    from prod.tag_address
    group by 1
) as c
on a.from_address = c.address
left join 
(
    select address
        ,max(tag_code) as tag_code
    from prod.tag_address
    group by 1
) as d
on a.to_address = d.address
where a.transaction_hash = '{txn_hash}'
order by log_index asc
""".format(txn_hash=txn_hash)
df = pd.read_sql(txn_detail_sql, con=connection)

/opt/conda/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


# Coinbase Transfer

In [5]:
def get_coinbase_trf(txn_list,start_block,end_block):
    sql ="""
    select a.block_number
        ,a.transaction_hash
        ,max(case when a.miner is not null then a.amount else 0 end) as coinbase_trf_amt
        ,max(b.effective_gas_price * b.gas_used / POWER(10.0, 18)) as gas_fee
    from (
        select a.block_number,a.transaction_hash
            ,a.from_address,a.to_address
            ,b.miner,a.token_address,a.symbol,a.amount
        from dw.dwb_token_transfer_detail_eth_hi as a
        left join prod.blocks as b
        on a.block_number = b.block_number
        and a.to_address = b.miner
        and b.block_number >= {start_block} and b.block_number <= {end_block}
        and b.`chain` = 'eth'
        where a.block_number >= {start_block} and a.block_number <= {end_block}
        and a.transaction_hash in ({txn_list})
    ) as a 
    left join prod.transactions_eth as b
    on a.transaction_hash = b.transaction_hash
    and b.block_number >= {start_block} and b.block_number<= {end_block}
    group by 1,2
    """.format(
        start_block = start_block,
        end_block = end_block,
        txn_list = ",".join('"'+txn+'"' for txn in txn_list)
    )
    df = getSqlResult(sql)
    return df


txn_list = [
    '0xf41775f2a03045511deee450845d6e197a2909ac46c2f3759f2dff699eee5aa1',
    '0xec7b3c821492dd76ac274b7ac37e0b55cabe535ecbd917b740f8d3389c2c3b27',
    '0xc154db3aefe3e7a1d58ee45c2b543faa49e341075588f7e1e113ffb93d74d930'
]
start_block = 15100000
end_block = 15178000
df = get_coinbase_trf(txn_list,start_block,end_block)
df

,0,1,2,3
0,15177999,0xec7b3c821492dd76ac274b7ac37e0b55cabe535ecbd9...,0.517402,0.005718
1,15177999,0xf41775f2a03045511deee450845d6e197a2909ac46c2...,0.231752,0.013493
2,15103060,0xc154db3aefe3e7a1d58ee45c2b543faa49e341075588...,0.000000,0.016140


# Protocal Liquidity

In [80]:
def get_protocol_liquidity(pair_list):
    sql ="""
    select a.account_address as protocol_address
        ,a.token_address
        ,a.`value` * b.price as usd_value
    from dw.dws_token_balance_eth as a
    left join
    (
        select token_address,max(price) as price
        from dw.dwb_token_price_eth_byday_di
        where dt >= DATE_SUB(now(),1)
        group by 1
    ) as b
    on a.token_address = b.token_address
    where a.account_address in ({pair_list_str})
    """.format(
        pair_list_str = ",".join('"'+p[0]+'"' for p in pair_list)
    )
    df = getSqlResult(sql)
    df.set_axis(["protocol_address","token_address","liquidity"],axis=1,inplace=True)

    df_raw=[]
    for i in range(len(pair_list)):
        for j in range(len(pair_list[i])-1):
            df_raw.append([pair_list[i][0],pair_list[i][j+1]])
    df1 = pd.DataFrame(df_raw)
    df1.set_axis(["protocol_address","token_address"],axis=1,inplace=True)

    df_res = pd.merge(
        df,df1,how='inner',
        left_on=['protocol_address','token_address'],
        right_on=['protocol_address','token_address']
    )
    df_res = df_res.groupby(['protocol_address']).sum().reset_index()
    return df_res


pair_list = [
    ('0x760166fa4f227da29ecac3bec348f5fa853a1f3c','0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2','0x0000000000085d4780b73119b644ae5ecd22b376'), 
]
df = get_protocol_liquidity(pair_list)
df

,protocol_address,liquidity
0,0x760166fa4f227da29ecac3bec348f5fa853a1f3c,151234.03069


# Swap Data is Null

In [18]:
mysql_host = "142.132.134.21"
mysql_prot = 9030
mysql_user = "dw"
mysql_pd = "a64imrJwqKZc9jiU!*aCYi.xnh9*TVhhBmZi"
mysql_db = "dw"
connection = pymysql.connect(host=mysql_host, user=mysql_user, password=mysql_pd, db=mysql_db, port=mysql_prot, charset='utf8')
cursor = connection.cursor()
logging.info("connection created")

def is_null_data(txn_list,min_blo,max_blo):
    sql ="""
    select transaction_hash
        ,max(1-(`regexp`(input,"{pattern}") or length(input) = 266)) as data_isnot_0x_v1
    from prod.traces_bsc
    where transaction_hash in ({txn_list_str})
    and block_number >= {min_blo}
    and block_number <= {max_blo}
    and input like '0x022c0d9f%'
    group by 1
    """.format(
        txn_list_str = ",".join('"'+p+'"' for p in txn_list),
        min_blo = min_blo,
        max_blo = max_blo,
        pattern = "0{62}80{65}$"
    )
    df = getSqlResult(sql)
    df.set_axis(["transaction_hash","data_is_0x"],axis=1,inplace=True)
    return df

txn_list = ['0xd27f50aaf535b9f612b3508a46f88949098f891f59e2a734533ee9f7ab40032e', '0x4da2316fffa5881fd1c550b3c1cf40fdf52cc6e4bd413331074b90723e0bd027', '0x7097a910ae18c0f8cbb0dd46bdbacf75252eb4f18c255698c0ac62e5304a8a0f', '0x2646d615c79209c77ab5a996eb55d92ebc562850747ebb8766d704271866fb7b', '0xa4f04fb66e036545a31da259b881e5fdd3ce4e6dbf58ead808e2cb98646b6d74', '0x8e7586fae53d6b64cd85fe5c44c6ded08a6b0bd13783cdb6cd5710d02d77a6cf', '0xa4f3046feb0964e8e7c7b35bd02eb4c9aa4f9f745a888dd09eebd0e3a1fa529d', '0x3d0a3733c2ca97d2639332f90677563868ac590a00d1e628a0182181023d9846', '0x26394e24de17e43b5dc9cf86203076437f7411db338bb102039e1b5bcf00f8d8', '0x5761d235d0e278791c7cd777d29e8e5b014a2c0f5219926b5cf5645d263bfabe', '0x40b498a209b0df4c46591b12d080b34067a16c564c297ff036192e6a134f177b', '0x53fcf3b805ff7baf40080b58bc0d36e7b6fad275df30f721d8d6d00737b21171', '0x15c73eb91006b820c6ee070d3a17881ef25440a8f186e85bba855d636e0abbcc', '0x6d9e5418d10490065f524769c319f913d5df4d67e0f398b7d80fa771d57c8453', '0x37d24d3cc23042112480cc85fe47d5144cdc27fecc2dd70b30e1ab4e08e8da0a', '0x3af10ba3c9669c807544ff01923b5bea684690be2c91ac530294b339b2f51046', '0xf8d0bcb915e9da9aeb17417d0ed8dea259280797dda4a6b49d837f266b683ed0', '0xd9e76d210c000b51e5ed71b21a16df985a45d9dbe1b8d7f5a9bbdb6cd56ad913', '0xbb934e7308ccb765ab5359c8e143f45f9f2e2f1758bb1ec9d87b92777206eb8d', '0x546a0bc7b5fb12c2143dc2358192614637233d5d1f9645477fa9847a573802f9', '0x4c563f8fb039e3ef7802649b36e71a88a8e3833a9dc170239fb46cc97af33572', '0x64e75c555ca76bc0e2d72dd36f34eb9baaffbd077aac45a680a8c812e1203ca3', '0xbadbe4bd49899bc7b89c138343b388cce65fab73bee2c11020897c57d18cd613', '0x36b53b5c898b43b5225ac6510543a6536d80311712d8f42a44f9394ff45ec68f', '0x83b288f0cfd34a5629db1bac258543559aabd211835dad93fe51aa0723e5bccd', '0x6dca4dff18655a72a85e876b69b90de2c00a02bd0f57db8c7aba1ef6f113b4c4', '0xae4bb29618c6580c37f3b073f2839e5fc8f2522cd2b57c05c07e3a968e714f13', '0x9ee12bf6b54467c1d12480230698cc26fcc56cb4c671e78b1489bee4d21ac29a', '0x5f2953d97790383ef1113c7a40bc51d2ea32e32894ca3c82c04c5ac97d298743', '0xacfadbd8d4e6d514106b276a412bb8d8e3c23a39f34c84c2461b629142a9673c', '0xbcce4fdd496248dab7a328e9fa5e92f3099bc063993fbe059108fc75130be5fa', '0x16beb8502fa03a955adb3f8833e1469da1fd57f3b0fedad08dfc9897c864c5a8', '0xc3dee1fdc488777cd5fa0561e925975782e3a9567d37ce1cb426516ba728a5ac', '0xad93191ad6273dd795183cb43ab37d02ec07fd50a9c73fc421dd4c25d8448ca4', '0x001c07bf19d8a469df108bf53bf5ce249b1c59ed7d26cccad0344dd3828a835a', '0x890819ddcf8fbbb40f69a968964606491bdc463740532ebf3ec5ce3b519e82a0', '0x13ddb2b573d702720b711a4f1f4844453864437317dba0d77c0115e44331d9e3', '0x3740638ad8f4b65593557f0af6ccbea17b74d8b933c30717e8c2a39bc22c77f0', '0x62e81cca30bead4101fb30dd7ab21581d15eb785eb8bb4276dfaf6442eac0229', '0x6e7199b5a43ed3c6bb4732ef16bf4aa30b9448b18803e8460df63c2234daed89', '0xcadb6c0634eb73a85cd862823ee5293ed86d661ca40605307f7f03466d201022', '0x315aaf1e6d68333a5b469b9e32de77bd20ddc48d04c9a4bebe8d8e52db618f87', '0xc1bb94bb9918ccd64715cc249d1a355a6dedb4db1fdcd76d551570369ce0360e', '0x2ea9af0d0e5c85a4e93709b87b3527188bd124d09c67c6aa90130f06c22330a9', '0x30adb5ab96ba8a7d6d16794187f5dd7fb1662de4363e64d6c34aa8ae28809ac5', '0x866c981f537ebbefa1eb5451a85cd43bcac3b5f6a4064271329a1dff521cccc4', '0x96ae22ecc4f9e1d38955737f9257c614c6125671cae215b5f01c431af999fc14', '0xbe18f5e3492f03be80604da65e0c6942cdf0849b03d7c5206ca446c271e69772', '0xf37ee4024c3532de1ba19e559c5d2050578f3957c3f7ce871541ba89305bddd5', '0xf5a3a0192fc19ba89e71b39700104ab25012c026ace6a9b7beba4d3e0b21c34b', '0x89cb1c2d3784b71ac3c2febb1521f189d2d52ac9f6496584dba1ded5377f3672', '0x2b8574c57450ae2d729fd2e8bfb9b3fe8217bdc135dcfd2a8712d712408c7e58', '0x90c500b7c917064faf7b346b784c7d0ead03aa45fd540210d84a61b9be205a17', '0xda2e44e0c95357a7df69c177357c6af54c4e4a46b855e8578d93c86656657a78', '0xf1fa15e7c46b9eda1b7391b51fd30adada6daf98bfa1eedc1353a7dc25b68f24', '0x6e5fdff39237237fa89d77a62cbde2769616f7c13ca8550bbea5bf3741eeede5', '0x1462f293a70e42155fb32348f25191a16479657d959cf7531eb13c5565552911', '0x1790eb921efb0bb9132a10cc3deee17c2b73beb70f767fcf4aaa8c9eb139d4d9', '0x06c9975aa89876c560eb704cc066dd21ae5d1b8059d17a8ea64a394a0e26bd5b', '0xe42010ad69667d96b3826513be01ae0551cfed6564b6365b802895046391514c', '0x7f88eba7027b15e542a4c19cf26020c2f5e5c438e54e2be9d592728be598ac02', '0xa0b588666e53a925f047be9a4d7a2c8411243e98e289f279fce97b62f1881ea4', '0x6e80ccae18bbd68fcf516ea47fa9e51e1a205e1956e467368886d9d15117c598', '0x2dec71cc642bacc67322c6620fc4873c07d66ef00c618018acee09ff6f424243', '0xfcfbebf0a1468ef6f69f3fbd2dc856b97a86c03845011197ab8afb5228f387d2', '0xe2e1d074fbd50b4643c6145780cc90b5135d20af432abfae6f94b06fe7a2dc6a', '0x9efc6eeff3ccdfda2e812659f89921200bce38db1c733ffc9e3bd42a91ddcac9', '0xf46e88dbeaf1866aed228e42cdf17abf347bfdb6dd525811c9b12dc57a989611', '0x220932133aeb7f5cdb45e9d44842fb56851052c43fda0c890b83349c4bdca327', '0x3fff159b24b61ac82e49245a9cb086ed02a06826de4f8271d2eada587d2685a5', '0x226c5f638700d9aaaf4c36897c0db2367df3978bf2cb79c23391456729eef2ea', '0x3f834bba86de54c7bf842534481ff62f5fb03e8bd3dc0d8d087d00c08f40c098', '0xa3d34658cae6fe94b3e6c7bb1a7504d3e3f721f63bc0c35c6bf5ed245ee21ca7', '0x6b64e9732f2859dd008951cd984c2eee73552ef60cb6609bd8680dcaec21c290', '0x548fa49513d25d747b19e2568a06092834e1cbd24e9a6e8dde332a62968761c2', '0xeda0d06d6d2b0810a464cb67096b951426c16178a5ed3aac5d3da839457a58e2', '0x1c18ad3c4a7e390009227a2ccd971e194e5668670abac6c6e4841d0ab19c730a', '0x418598126372eb4db6db37fcd07ed725fd1fe5735f406ccc6d9362d5504e0ffd', '0x38315ed99e1a6eae1c70a8bbfb0621ad8611a3f5f394d1c315143407c1fefb11', '0xe74d75118a872ab0c6f4a000a9bf42a0642bc18147aedef8f008dbb4bbc07dbf', '0x807d776232228fe4a131cec5a831bbed300e3601a2cc685cf693ed855b0f62ba', '0xf2368ebe65d97eaf35cf85287f875d1d0967bcbb5172d952e3063b5325cc9f6a', '0x67d6d16a252ca0218bd0b4a44a410f401aaefbfa8b6909239926e9d8a46b362c', '0x5ad57520babc6637f6fe24f55ae73a62ce0356e8ce57133515ba4f4627fb5393', '0x9de3c7a7dd5d3b0fee98f8f7d8ea459b15657ccde114f0eff6b83e1eaa9f4214', '0x341d89f6e4b8f86b6fb6ba3f04c9c463553a2340f8829ff7a146f76b5491d5ef', '0x2c9d264ae0965287e6895a38052b256a72182edc742c070b9486b62bab7d5505', '0xbcd43f068a0cdf03c9d31f7e532f099cec9e7eab70bc81022973ca32e5cf8a0d', '0x9a14c1a963428951a076062274ccbaff48461689818202c885a0a9730ba8d343', '0x60fd6efc3b66f3ff93a3b11f288e591c9538868f895caa779bac630ac73cba1b', '0x3b623ed7534ae9538f18718fd3bfedf44620f7bd54ec9261491be5fae7b30aef', '0xdf1df8d54610eab5086b9b67e0068cbebb244048fedd9cc1d549571f1320059a', '0xd361215eb0dd518345feccb100c144f99f774e7d95f1cbf186aca5179a407f98', '0x46f208a4fcf5eb6fee3fac7aab22c77d4b46c13b09d650f39c3b4d2a841c1ffa', '0x065881481a646c06e236e9a0cd1f67349dbc4fab0dd312d397107ad624252b07', '0x402c6af6e7903035e8ce9265fcf5adabb48596cdcfea0a9de83560cf0130e08f', '0xbc8bd1859e10c93a3804fd23c6fbbb8c3fd0fe67dca12f827e1d147862a1cd1f', '0x3f4bc7642df393923497a505fda42102b0afc6f888629ff18731c46483151ab3', '0xab28cce933934c9474bc8b1a366aee712ef401ecf26350b3550f0da5655fa663', '0xb44572101806e468087d9412c64fa064042e299cf63fc54250b50409fd92eb20']
print(len(txn_list))
min_blo = 22239268
max_blo = 22243300
df = is_null_data(txn_list,min_blo,max_blo)
df



INFO:root:connection created


100


,transaction_hash,data_is_0x
0,0x341d89f6e4b8f86b6fb6ba3f04c9c463553a2340f882...,0
1,0xbe18f5e3492f03be80604da65e0c6942cdf0849b03d7...,1
2,0x37d24d3cc23042112480cc85fe47d5144cdc27fecc2d...,0
3,0x001c07bf19d8a469df108bf53bf5ce249b1c59ed7d26...,0
4,0xab28cce933934c9474bc8b1a366aee712ef401ecf263...,0
...,...,...
95,0xc3dee1fdc488777cd5fa0561e925975782e3a9567d37...,0
96,0x67d6d16a252ca0218bd0b4a44a410f401aaefbfa8b69...,0
97,0x40b498a209b0df4c46591b12d080b34067a16c564c29...,0
98,0x9efc6eeff3ccdfda2e812659f89921200bce38db1c73...,0


# Test

In [68]:
pair_list = [
    ('0x5777d92f208679db4b9778590fa3cab3ac9e2168','0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', '0x6b175474e89094c44da98b954eedeac495271d0f'),
    ('0x2db094c50181f39863858f1878545feb26b975d0', '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', '0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e'), 
    ('0x3416cf6c708da44db2624d63ea0aaef7113527c6', '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', '0xdac17f958d2ee523a2206206994597c13d831ec7'), 
    ('0xd8dec118e1215f02e10db846dcbbfe27d477ac19', '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2', '0x6b175474e89094c44da98b954eedeac495271d0f'), 
    ('0xe6ff8b9a37b0fab776134636d9981aa778c4e718', '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2', '0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599'), 
    ('0xb20ed18ee19747c0811d57d38b7ab3d71e44c191', '0x6b175474e89094c44da98b954eedeac495271d0f', '0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9'), 
    ('0x48da0965ab2d2cbf1c17c09cfb5cbe67ad5b1406', '0x6b175474e89094c44da98b954eedeac495271d0f', '0xdac17f958d2ee523a2206206994597c13d831ec7')
]
df_raw=[]
for i in range(len(pair_list)):
    for j in range(2):
        df_raw.append([pair_list[i][0],pair_list[i][j+1]])
df1 = pd.DataFrame(df_raw)
df1.set_axis(["protocol_address","token_address"],axis=1,inplace=True)

df_res = pd.merge(df,df1,how='inner',left_on=['protocol_address','token_address'],right_on=['protocol_address','token_address'])
df_res.groupby(['protocol_address']).sum().reset_index()

,protocol_address,market_value
0,0x2db094c50181f39863858f1878545feb26b975d0,9.003840e-06
1,0x3416cf6c708da44db2624d63ea0aaef7113527c6,1.614799e+08
2,0x48da0965ab2d2cbf1c17c09cfb5cbe67ad5b1406,7.680899e+05
3,0x5777d92f208679db4b9778590fa3cab3ac9e2168,8.981315e+08
4,0xb20ed18ee19747c0811d57d38b7ab3d71e44c191,0.000000e+00
5,0xd8dec118e1215f02e10db846dcbbfe27d477ac19,7.531073e+00
6,0xe6ff8b9a37b0fab776134636d9981aa778c4e718,3.507081e+02


In [89]:
int('027f7d0bdb920000',16)/pow(10,18)

0.18